# Extracting Flooding Areas from NRT Sentinel-1 Satellite Imagery

In [1]:
import pandas as pd            # ready geospatial data
import os
import subprocess              # for multithreading
import multiprocessing         #  "     "      "
from concurrent.futures import ThreadPoolExecutor
from ipyleaflet import Map, basemaps, basemap_to_tiles 
from ipyleaflet import GeoData, LayersControl, ImageOverlay
import geopandas as gpd
from sidecar import Sidecar
import ipywidgets as widgets     # interactive display

In [2]:
"""
These files were obtained using the ASF Vertex Data Search.
They were submitted for Radio Terrain Correctiom.
radiometry: gamma0
scale: decibel
pixel spacing: 10m
"""

box_data = {
    'zipfile': ['S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A.zip',
                'S1A_IW_20250217T233955_DVP_RTC10_G_gdufem_E701.zip'],
    'link': ['https://ky.box.com/shared/static/xwhzpb6entefdhsi8jfhgii1f4ehh6x6.zip', 
             'https://ky.box.com/shared/static/81wf3fabzhzsux29nhe4ojdaq0qtwp89.zip']
        }
df = pd.DataFrame(data=box_data)

base_scene = df.at[0,'zipfile']
base_dir = os.path.splitext(base_scene)[0]
flood_scene = df.at[1, 'zipfile']
flood_dir = os.path.splitext(flood_scene)[0]

print(f'Base Image: {base_dir}\nFlood Image: {flood_dir}')

Base Image: S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A
Flood Image: S1A_IW_20250217T233955_DVP_RTC10_G_gdufem_E701


Function to download files from Box\* \*\*

\* At first I create a function to use the Box download link, but it was corrupting the file and not letting me unzip.  As a last resort, I'm using subprocess.

\*\* If all else fails, the download links can be used to directly download files and unzip locally

---
## Downloading

In [3]:
def box_download(df):
    out_folder = "data"
    download_zip = os.path.join(out_folder, 'file.zip')
    for row_index, row in df.iterrows():
        link = row['link']
        unzipped_folder = os.path.splitext(row['zipfile'])[0]  # get basename of zip file
        unzip_path = os.path.join(out_folder, unzipped_folder)

        cmd = f"curl -L -o {download_zip} {link} && unzip {download_zip} -d {out_folder}"
        
        if not os.path.exists(unzip_path):
            subprocess.run(cmd, shell=True, check=True)
            # delete downloaded zip file.
            os.remove(download_zip)
        else:
            print(f'Extracted Files already exist for "{unzipped_folder}"')

I don't think using multiprocessing helps now that I'm using a subprocess, but it's not hurting it either.

Download could take several minutes, depending upon bandwidth.  Zipped up, the scenes are over 10gb in size each.

In [4]:
logical_cores = multiprocessing.cpu_count()
# determine number of threads to use for multiprocessing
num_workers = int(logical_cores * 0.75)  # rounds down in case not a whole number
print(f'Number of threads to use: {num_workers}')

with ThreadPoolExecutor(max_workers=num_workers) as executor:
    executor.map(box_download(df))

Number of threads to use: 7
Extracted Files already exist for "S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A"
Extracted Files already exist for "S1A_IW_20250217T233955_DVP_RTC10_G_gdufem_E701"


#### Review the README.md.txt file

The folders contain several files.  Each image (png or tif) has a corresponding metadata (xml) file.  The folder also includes a shapefile (\*_shape.\*) and a google kmz which display a geometry on a map.  There is also a README.md.txt file in each folder that breaks down how to undertand the file name and what is included in the scene folder.  Try 
removing the *.txt* and open with a Markdown Viewer.  

Let's take a look at some of the files to get an idea of what we're looking at.

In [5]:
# read in base image shapefile
base_shapefile = (f'data/{base_dir}/{base_dir}_shape.shp')

# read in shapefile to geodataframe
gdf_base_shapefile = gpd.read_file(base_shapefile)
print(f'source projection: {gdf_base_shapefile.crs}')

# redefine the projection to work with the map
gdf_base_shapefile = gdf_base_shapefile.to_crs('EPSG:4326')
print(f'redefined projection: {gdf_base_shapefile.crs}')

source projection: EPSG:32616
redefined projection: EPSG:4326


In [6]:
# read in geodataframe to a ipyleaflet GeoData
base_geo_data = GeoData(geo_dataframe = gdf_base_shapefile,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.1, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'base Geometry')

In [7]:
m = Map(
    basemap=basemap_to_tiles(basemaps.CartoDB.Positron),
    center=(38, -85.5),
    zoom=7.3
)
m.add(base_geo_data)
# m.add(LayersContol())
m


Map(center=[38, -85.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

## TODO

Add a section that creates a grid to display several images side by side

In [10]:
# display RGB Composite File
rgb_png = './data/S1A_IW_20250217T233955_DVP_RTC10_G_gdufem_E701/S1A_IW_20250217T233955_DVP_RTC10_G_gdufem_E701_rgb.png'
file = open(rgb_png, "rb")
image = file.read()
image_widget = widgets.Image(
    value=image,
    format='png',
    width=300,
    height=400,
)
print("\nNote how we can see the Ohio River starting near the uppermost right corner,\nextending to middle on the left.  This is the derived RGB image.  It will\nnot be used for analysis.")
display(image_widget)


Note how we can see the Ohio River starting near the uppermost right corner,
extending to middle on the left.  This is the derived RGB image.  It will
not be used for analysis.


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x08\x00\x00\x00\x05\xfd\x08\x02\x00\x00\x00\xf5\x1e/…

---
## Processing